<a href="https://colab.research.google.com/github/Kazuya-Y/public/blob/main/Kazuya_%E3%83%87%E3%83%BC%E3%82%BF%E5%88%86%E6%9E%90.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [44]:
import pandas as pd
df_monthly_test_results=pd.read_csv('/content/monthly_test_results_20260127_210102.csv')
df_study_time_monthly=pd.read_csv('/content/study_time_monthly_20260127_210031.csv')

df_monthly_test_results['テスト種類']
col_needed=['受講生ID','受講生名','実施日','テスト種類','総合']
df_monthly_test_results=df_monthly_test_results[col_needed]

def versant_score_to_cefr(score):
    if 10 <= score <= 29:
        return 'A1'
    elif 30 <= score <= 42:
        return 'A2'
    elif 43 <= score <= 58:
        return 'B1'
    elif 59 <= score <= 75:
        return 'B2'
    elif 76 <= score <= 84:
        return 'C1'
    elif 85 <= score <= 90:
        return 'C2'
    else:
        return None

def versant_sl_score_to_cefr(score):
    if  score <= 35:
        return 'A1'
    elif 36 <= score <= 46:
        return 'A2'
    elif 47 <= score <= 57:
        return 'B1'
    elif 58 <= score <= 68:
        return 'B2'
    elif 69 <= score <= 78:
        return 'C1'
    elif 79 <= score <= 80:
        return 'C2'
    else:
        return None

def casec_score_to_cefr(score):
    if 0 <= score <= 399:
        return 'A1'
    elif 400 <= score <= 599:
        return 'A2'
    elif 600 <= score <= 799:
        return 'B1'
    elif 800 <= score <= 899:
        return 'B2'
    elif 900 <= score <= 1000:
        return 'C1'
    else:
        return None

def cefr_to_num(cefr):
    if cefr == 'A1':
        return 1
    elif cefr == 'A2':
        return 2
    elif cefr == 'B1':
        return 3
    elif cefr == 'B2':
        return 4
    elif cefr == 'C1':
        return 5
    elif cefr == 'C2':
        return 6
    else:
        return None

mask = df_monthly_test_results['テスト種類'] == 'Versant'
df_monthly_test_results.loc[mask, 'CEFR'] = df_monthly_test_results.loc[mask, '総合'].apply(versant_score_to_cefr)

mask_sl = df_monthly_test_results['テスト種類'] == 'Versant Speaking & Listening'
df_monthly_test_results.loc[mask_sl, 'CEFR'] = df_monthly_test_results.loc[mask_sl, '総合'].apply(versant_sl_score_to_cefr)

mask_casec = df_monthly_test_results['テスト種類'] == 'CASEC'
df_monthly_test_results.loc[mask_casec, 'CEFR'] = df_monthly_test_results.loc[mask_casec, '総合'].apply(casec_score_to_cefr)

df_monthly_test_results.head(30)

,受講生ID,受講生名,実施日,テスト種類,総合,CEFR
0,55,嘉門延親,2021-01-26,Versant,42,A2
1,55,嘉門延親,2021-03-04,Versant,46,B1
2,55,嘉門延親,2021-04-07,Versant,44,B1
3,55,嘉門延親,2021-05-11,Versant,46,B1
4,55,嘉門延親,2021-06-07,Versant,47,B1
5,55,嘉門延親,2021-07-09,Versant,47,B1
6,55,嘉門延親,2021-08-05,Versant,51,B1
7,55,嘉門延親,2021-09-15,Versant,50,B1
8,55,嘉門延親,2021-10-13,Versant,48,B1
9,55,嘉門延親,2021-11-12,Versant,51,B1


In [45]:
df_casec_s=pd.read_csv('/content/CASEC_Speaking_CEFR.csv')
df_temp_casec_s=df_monthly_test_results[df_monthly_test_results['テスト種類']=='CASEC Speaking']
df_temp_non_casec_s=df_monthly_test_results[df_monthly_test_results['テスト種類']!='CASEC Speaking']

df_temp_casec_s = df_temp_casec_s.merge(
    df_casec_s,
    on=["受講生名", "総合"],
    how="left",
    suffixes=("", "_casec")
)

df_temp_casec_s = df_temp_casec_s.drop(columns=["CEFR"])
df_temp_casec_s = df_temp_casec_s.rename(columns={"CEFR_casec": "CEFR"})

df_monthly_test_results = pd.concat([df_temp_non_casec_s, df_temp_casec_s], ignore_index=True)
df_monthly_test_results['CEFR_score']=df_monthly_test_results['CEFR'].apply(cefr_to_num)

In [46]:
df_monthly_test_results

,受講生ID,受講生名,実施日,テスト種類,総合,CEFR,CEFR_score
0,55,嘉門延親,2021-01-26,Versant,42,A2,2.0
1,55,嘉門延親,2021-03-04,Versant,46,B1,3.0
2,55,嘉門延親,2021-04-07,Versant,44,B1,3.0
3,55,嘉門延親,2021-05-11,Versant,46,B1,3.0
4,55,嘉門延親,2021-06-07,Versant,47,B1,3.0
...,...,...,...,...,...,...,...
4000,833,石川 玲奈,2024-05-17,CASEC Speaking,75,A1,1.0
4001,1015,石原 久美子,2025-04-06,CASEC Speaking,140,A2,2.0
4002,1048,小山 泰嵩,2025-04-12,CASEC Speaking,150,A2,2.0
4003,1048,小山 泰嵩,2025-08-03,CASEC Speaking,170,B1,3.0


In [24]:
from google.colab import files
df_test=df_monthly_test_results[df_monthly_test_results['テスト種類']=='CASEC Speaking'].head(30)
df_test.to_csv("data.csv", index=False, encoding="utf-8-sig")
files.download("data.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [48]:
# 実施日を datetime に変換
##df_monthly_test_results["実施日"] = df_monthly_test_results.to_datetime(df["実施日"])

# 受講生IDごとに「最も早い実施日」の CEFR_score を取得
first_scores = (
    df_monthly_test_results.loc[df_monthly_test_results.groupby("受講生ID")["実施日"].idxmin()]
    .set_index("受講生ID")["CEFR_score"]
)

# score_first 列として元の DataFrame に追加
df_monthly_test_results["score_first"] = df_monthly_test_results["受講生ID"].map(first_scores)

# 受講生IDごとに「最も遅い実施日」の CEFR_score を取得
last_scores = (
    df_monthly_test_results.loc[df_monthly_test_results.groupby("受講生ID")["実施日"].idxmax()]
    .set_index("受講生ID")["CEFR_score"]
)

# score_first 列として元の DataFrame に追加
df_monthly_test_results["score_last"] = df_monthly_test_results["受講生ID"].map(last_scores)

df_monthly_test_results.head(50)
##df_monthly_test_results['CEFR_score'].isna().sum()
##df_monthly_test_results[df_monthly_test_results['CEFR_score'].isna()]

,受講生ID,受講生名,実施日,テスト種類,総合,CEFR,CEFR_score,score_first,score_last
0,55,嘉門延親,2021-01-26,Versant,42,A2,2.0,2.0,3.0
1,55,嘉門延親,2021-03-04,Versant,46,B1,3.0,2.0,3.0
2,55,嘉門延親,2021-04-07,Versant,44,B1,3.0,2.0,3.0
3,55,嘉門延親,2021-05-11,Versant,46,B1,3.0,2.0,3.0
4,55,嘉門延親,2021-06-07,Versant,47,B1,3.0,2.0,3.0
5,55,嘉門延親,2021-07-09,Versant,47,B1,3.0,2.0,3.0
6,55,嘉門延親,2021-08-05,Versant,51,B1,3.0,2.0,3.0
7,55,嘉門延親,2021-09-15,Versant,50,B1,3.0,2.0,3.0
8,55,嘉門延親,2021-10-13,Versant,48,B1,3.0,2.0,3.0
9,55,嘉門延親,2021-11-12,Versant,51,B1,3.0,2.0,3.0
